In [1]:
import numpy as np
import matplotlib.pyplot as plt
import vigra
from vigra import graphs
import copy
import os
import pylab
import time
from scipy.io import loadmat
import sys
import shutil

import inferno
import multicutAuxFunctions as maf

In [2]:
resultsPath = 'results/151211_weightsIn[0,1]_ph_w\oRf_w\oConstr_voi_w\RF_w\oConstrOnRF_origGradDirSearch/'

################################### Training Data ########################################

trainSetPath = 'trainingSet/'
#trainSetPath = 'fewImages/'


path = os.walk(trainSetPath)

trainingIds = []
trainingImgs = []

trainingGtLabels = []
trainingGtSols = []

trainingRags = []

superpixelDiameter = 20      # super-pixel size
slicWeight         = 25.0    # SLIC color - spatial weight
nodeNumStop        = 50      # desired num. nodes in result
minSize            = 15

print "Loading Training Data..."
############# load images and convert to LAB #############
for root, dirs, files in path:
    jpgFiles = [filename for filename in files if filename.endswith('.jpg')]
    T = len(jpgFiles)
    for i, filename in enumerate(jpgFiles):
        
        sys.stdout.write('\r')
        sys.stdout.write("[%-50s] %d%%" % ('='*int(float(i+1)/T*50), int(float(i+1)/T*100)))
        sys.stdout.flush()

        fileId = filename[:-4]
        trainingIds.append(fileId)
        
        img = vigra.impex.readImage(root + '/' + filename)
        trainingImgs.append(img)
        imgLab = vigra.colors.transform_RGB2Lab(img)
            
        gridGraph = graphs.gridGraph(img.shape[0:2])

        slicLabels = vigra.analysis.labelImage(vigra.analysis.slicSuperpixels(imgLab, slicWeight, superpixelDiameter, minSize=minSize)[0])
        rag = graphs.regionAdjacencyGraph(gridGraph, slicLabels)
        trainingRags.append(rag)
        
        #gtWatershed = loadmat('trainingSet/groundTruth/' + fileId + '.mat')['groundTruth'][0,0][0][0][0]
        #gtLabel = maf.getSuperpixelLabelList(rag, gtWatershed)
        
        gtLabel = np.load('groundTruthLabels/' + fileId + '.npy')


        trainingGtLabels.append(gtLabel)
        trainingGtSols.append(maf.getGroundTruthSol(rag, gtLabel))
        
        
        
### Feature Spaces
# Training

#testingFeatureSpacesPath = resultsPath + 'featureSpaces/training/'

trainingFeatureSpaces = []
trainingEdges = []
t1 = time.time()
T = len(trainingImgs)
print "\nBuilding up Training Feature Space..."
sys.stdout.flush()
for i, (rag, img, trainId) in enumerate(zip(trainingRags, trainingImgs, trainingIds)):
    trainingEdges.append(rag.uvIds().astype('uint64'))

    #if (os.path.isfile(testingFeatureSpacesPath + trainId + '.npy') == True):
    #    features = np.load(testingFeatureSpacesPath + trainId + '.npy')
    #    if (os.path.isfile(resultsPath + 'featureSpaces/featureNames.npy') == True):
    #        featureNames = list(np.load(resultsPath + 'featureSpaces/featureNames.npy'))
        
    #else:
    #    features, featureNames = maf.getFeatures(rag, img, trainId)
    #    np.save(testingFeatureSpacesPath + trainId + '.npy', features)
    #    np.save(resultsPath + 'featureSpaces/featureNames.npy', featureNames)
        
    features = np.load('featureSpaces/full/' + trainId + '.npy')
    featureNames = list(np.load('featureSpaces/full/featureNames.npy'))


    sys.stdout.write('\r')
    sys.stdout.write("[%-50s] %d%%" % ('='*int(float(i+1)/T*50), int(float(i+1)/T*100)))
    sys.stdout.flush()

    trainingFeatureSpaces.append(features)
t2 = time.time()

print "\nTime to built up Training Feature Space:", t2-t1, "sec"




########################## Norm Feature Spaces to [-1, 1] ############################################
'''
print "Start to change Norm of Feature Spaces to [-1, 1]..."
t1 = time.time()
for n in range(len(trainingFeatureSpaces)):
    featureSpace = trainingFeatureSpaces[n]
    for edgeWeights in featureSpace.transpose():
        edgeWeights *= 2
        edgeWeights -= 1

t2 = time.time()

print "Time to change norm on Feature Spaces: ", t2-t1
'''


### Load Previous partition Hamming Weights
print "Load previous partitionHammingWeights"
auxWeightVector = np.load(resultsPath + 'partitionHamming/weights.npy')
weightVector = inferno.learning.WeightVector(auxWeightVector.shape[0], 0.0)
for n in range(len(weightVector)):
    weightVector[n] = auxWeightVector[n]
    
    
os.makedirs(resultsPath + 'partitionHamming/trainingResults/segmentedImages/')
maf.performTesting2(trainingImgs, trainingRags, trainingEdges, trainingFeatureSpaces, trainingIds, trainingGtLabels,
                    featureNames, weightVector, resultsPath + 'partitionHamming/trainingResults/')
    


########################## Add Random Forest Feature ###################################


print "Loading Random Forest..."
RF = vigra.learning.RandomForest('featureSpaces/full/RF.hdf5')
print "Get probabilities from Random Forest..."
trainingRfProbs = maf.getProbsFromRF(trainingFeatureSpaces, RF)
trainingFeatureSpaces[:] = [np.concatenate((featureSpace, (prob[:,1]).reshape(prob.shape[0],1)), axis=1) for featureSpace, prob in zip(trainingFeatureSpaces, trainingRfProbs)]


featureNames.append('RF_Prob')
nFeatures = trainingFeatureSpaces[0].shape[1]


########################## Norm Feature Spaces to [-1, 1] (just RF Feature) ############################################
'''
print "Start to change Norm of Feature Spaces to [-1, 1]..."
t1 = time.time()
for n in range(len(trainingFeatureSpaces)):
    featureSpace = trainingFeatureSpaces[n]
    edgeWeightsRF = featureSpace.transpose()[-1]
    edgeWeightsRF *= 2
    edgeWeightsRF -= 1


t2 = time.time()

print "Time to change norm on Feature Spaces: ", t2-t1

'''

    
    
############################ Stochastic Gradient Learner (Variation of Information) ##########################

# Load Previous VOI Weights
print "Loading previous VOI weights"
auxWeightVector = np.load(resultsPath + 'VOI/weights.npy')
weightVector = inferno.learning.WeightVector(auxWeightVector.shape[0], 0.0)
for n in range(len(weightVector)):
    weightVector[n] = auxWeightVector[n]
    
os.makedirs(resultsPath + 'VOI/trainingResults/segmentedImages/')
maf.performTesting2(trainingImgs, trainingRags, trainingEdges, trainingFeatureSpaces, trainingIds, trainingGtLabels,
                    featureNames, weightVector, resultsPath + 'VOI/trainingResults/')

Loading Training Data...
[==================================================] 100%
Building up Training Feature Space...
[==================================================] 100%
Time to built up Training Feature Space: 0.181408166885 sec
Load previous partitionHammingWeights
Loading Random Forest...
Get probabilities from Random Forest...
Loading previous VOI weights


/usr/lib/pymodules/python2.7/matplotlib/pyplot.py:412: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_num_figures`).
  max_open_warning, RuntimeWarning)


Load previous partitionHammingWeights


NameError: name 'resultsPath' is not defined